In [1]:
import pylhe
import awkward as ak
import pandas as pd

def extract_particle_data(lhe_file, particle_ids):
    """
    Extracts data for specified particles from the LHE file.

    Parameters:
        lhe_file (str): Path to the LHE file.
        particle_ids (list of int): List of particle IDs to extract.

    Returns:
        pd.DataFrame: DataFrame containing the extracted particle data.
    """
    # Read the LHE file and convert to an Awkward Array
    events = pylhe.read_lhe_with_attributes(lhe_file)
    arr = pylhe.to_awkward(events)

    # Filter particles based on the specified IDs
    mask = ak.any(arr.particles.id[:, :, None] == ak.Array(particle_ids), axis=2)
    filtered_particles = arr.particles[mask]

    # Extract relevant attributes
    particle_data = {
        'event': ak.broadcast_arrays(ak.arange(len(filtered_particles)), filtered_particles.id)[0],
        'id': filtered_particles.id,
        'px': filtered_particles.vector.x,
        'py': filtered_particles.vector.y,
        'pz': filtered_particles.vector.z,
        'E': filtered_particles.vector.t,
        'status': filtered_particles.status,
        'helicity': filtered_particles.spin
    }

    # Convert to a Pandas DataFrame
    df = pd.DataFrame({key: ak.to_numpy(value) for key, value in particle_data.items()})
    return df


In [3]:
# Path to your LHE file
lhe_file = r"C:\Users\gabri\Documents\University\Year 4\Mphys\DataFiles\unweighted_events.lhe"


# Extract data for specific particles
z_data = extract_particle_data(lhe_file, particle_ids=[23])




TypeError: __init__() missing 1 required positional argument: 'dtype'